In [1]:
import pandas as pd
import numpy as np
import csv
import sys
import os
import glob

In [2]:
def get_attri(path,im_w_workers,im_w_data,il,fbl):
    df = pd.read_csv (path)
    RawFile = path
    Keywords = ['Answer.voc_emotic_ava','Answer.facebook_profile','Answer.EE371R','Answer.blur_dataset']
    with open(RawFile) as infile:
        reader = csv.reader(infile)
        names=next(reader)
        workers = df.loc[:,'WorkerId'].values.tolist()
        visited_names = []
        if len(im_w_workers)!=0:
            for i in range(len(im_w_workers)):
                visited_names.append(im_w_workers[i][0])
        index = 0
        for name in names:
            for keyw in Keywords:
                if keyw in name and ' ' not in name and '+' not in name and 'listmyfiles.txt' not in name and 'pixels.zip' not in name and 'circle_radii' not in name and 'facebook_profile/25213.jpg' not in name:
                    name1 = name.replace('Answer.','')
                    list_val = df.loc[:,name].values.tolist()
                    index_to_pop = []
                    flag = -1
                    find_index = 0
                    if name in visited_names:
                        flag = 0
                        index = visited_names.index(name)
                        find_index = index
                        pre_workers = im_w_workers[index]
                        pre_workers = pre_workers[1:]
                        for w in workers:
                            if w in pre_workers:
                                index_to_pop.append(workers.index(w))
                            else:
                                im_w_workers[index].append(w)
                    else:
                        tmp = []
                        tmp.append(name)
                        tmp = tmp + workers
                        im_w_workers.append(tmp)
                    item_to_del = []
                    for i in index_to_pop:
                        item_to_del.append(list_val[i])
                    for i in item_to_del:
                        list_val.remove(i)
                    list_val_new = list_val
                    if '%20' in name1:
                        name1 = name1.replace('%20', ' ')
                    if '%2B' in name1:
                        name1 = name1.replace('%2B', '+')
                    if flag == -1:
                        tmp = []
                        tmp.append(name1)
                        tmp = tmp + list_val_new
                        im_w_data.append(tmp)
                    elif flag == 0:
                        #for l in list_val_new:
                        im_w_data[find_index] = im_w_data[find_index] + list_val_new 

In [8]:
files = 'worker_reasons_image/*.csv'
im_l = 'i_used_in_model.csv'
fb_l = 'fbpro.csv'
inf = csv.reader(open(im_l, "r+"), delimiter = ',')
inf2 = csv.reader(open(fb_l, "r+"), delimiter = ',')
il = []
fbl = []
im_w_workers = []
im_w_data = []
im_w_fdata = []
for row in inf:
    il.append(row[0])
for row in inf2:
    fbl.append('facebook_profile/'+row[0])
for path in glob.glob(files):
    file_name = path.split('/')
    n = len(file_name)
    file_name = file_name[n-1]
    get_attri(path,im_w_workers,im_w_data,il,fbl)
for d in im_w_data:
    if d[0] in il and d[0] not in fbl:
        im_w_fdata.append(d)
for d in im_w_fdata:
    if 'EE371R/rqk63-85.jpg' in d[0]:
        d[0] = d[0].replace('EE371R/rqk63-85.jpg', 'EE371R/rq85.jpg')
    elif 'EE371R/rqk63-59.jpg' in d[0]:
        d[0] = d[0].replace('EE371R/rqk63-59.jpg', 'EE371R/rq59.jpg')
    elif 'EE371R/rqk63-73.jpg' in d[0]:
        d[0] = d[0].replace('EE371R/rqk63-73.jpg', 'EE371R/rq73.jpg')
    row.append(d[0])
name = []
oe_count = []
ue_count = []
grainy_count = []
blurry_count = []
none_count = []
for l in im_w_fdata:
    name.append(l[0])
    l = l[1:]
    oe = 0
    ue = 0
    grainy = 0
    blurry = 0
    none = 0
    for ele in l:
        if ele == "oe":
            oe +=1
        elif ele == "ue":
            ue +=1
        elif ele == "grainy":
            grainy +=1
        elif ele == "blurry":
            blurry +=1
        elif ele == "none":
            none +=1
        elif "|" in ele:
            c = ele.split("|")
            c1 = c[0].replace("  ", "")
            c2 = c[1].replace("  ", "")
            if c1 == "oe":
                oe +=1
            elif c1 == "ue":
                ue +=1
            elif c1 == "grainy":
                grainy +=1
            elif c1 == "blurry":
                blurry +=1
            elif c1 == "none":
                none +=1
            if c2 == "oe":
                oe +=1
            elif c2 == "ue":
                ue +=1
            elif c2 == "grainy":
                grainy +=1
            elif c2 == "blurry":
                blurry +=1
            elif c2 == "none":
                none +=1
    oe_count.append(oe)
    ue_count.append(ue)
    grainy_count.append(grainy)
    blurry_count.append(blurry)
    none_count.append(none)

In [11]:
df = pd.DataFrame()
df['name'] = name
df['#oe'] = oe_count
df['#ue'] = ue_count
df['#grainy'] = grainy_count
df['#blurry'] = blurry_count
df['none'] = none_count

In [13]:
df.to_csv('attributes_image.csv')